In [23]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
from os import walk

In [24]:
spark = SparkSession.builder \
.master("local") \
.appName("join_groupby_df") \
.getOrCreate()

In [48]:
empdf = spark.read \
.format("json") \
.option("inferSchema",True) \
.option("path", "D:\BigData\Week12- Apache Spark - Structured API Part-2\employee.json") \
.load()

In [49]:
deptdf = spark.read \
.format("json") \
.option("inferSchema", True) \
.option("path","D:\BigData\Week12- Apache Spark - Structured API Part-2\dept.json") \
.load()

In [88]:
df1 = empdf.join(deptdf, empdf.deptid==deptdf.deptid, "right")
df2 = df1.drop(empdf.deptid)
df2.show()

+--------------------+----+-------+----+------+---------+------+
|             address| age|empname|  id|salary| deptName|deptid|
+--------------------+----+-------+----+------+---------+------+
|[[Pune, Maharasht...|  25| satish|1201|  5000|       IT|    11|
|[[Pune, Maharasht...|  28|krishna|1202|  6000|       HR|    21|
|[[Pune, Maharasht...|  39|  amith|1203|  7000|Marketing|    31|
|[[Pune, Maharasht...|  23| prudvi|1205|  9000|      Fin|    41|
|[[Pune, Maharasht...|  23|  javed|1204|  8000|      Fin|    41|
|                null|null|   null|null|  null|    Admin|    51|
+--------------------+----+-------+----+------+---------+------+



In [96]:
df2.groupBy("deptid") \
.agg(count("empname").alias("empcount"),first("deptName").alias("deptName")).show()

+------+--------+---------+
|deptid|empcount| deptName|
+------+--------+---------+
|    31|       1|Marketing|
|    51|       0|    Admin|
|    41|       2|      Fin|
|    11|       1|       IT|
|    21|       1|       HR|
+------+--------+---------+



In [62]:
empdf.createOrReplaceTempView("employee")
deptdf.createOrReplaceTempView("department")

In [66]:
spark.sql("""select d.deptName, d.deptid, count(e.id) as empcount from 
employee e right join department d on e.deptid=d.deptid group by d.deptid, d.deptName""").show()

+---------+------+--------+
| deptName|deptid|empcount|
+---------+------+--------+
|       HR|    21|       1|
|    Admin|    51|       0|
|Marketing|    31|       1|
|       IT|    11|       1|
|      Fin|    41|       2|
+---------+------+--------+

